In [40]:
import requests
import defs
import pandas as pd

In [41]:
session = requests.Session()

In [42]:
ins_df = pd.read_pickle("instruments.pkl")

In [43]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD']

In [44]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [45]:
code, res = fetch_candles("EUR_USD",9,"H1")

In [46]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 9820,
   'time': '2024-09-27T12:00:00.000000000Z',
   'bid': {'o': '1.11617', 'h': '1.12010', 'l': '1.11534', 'c': '1.11990'},
   'mid': {'o': '1.11625', 'h': '1.12018', 'l': '1.11541', 'c': '1.11998'},
   'ask': {'o': '1.11633', 'h': '1.12026', 'l': '1.11548', 'c': '1.12006'}},
  {'complete': True,
   'volume': 9038,
   'time': '2024-09-27T13:00:00.000000000Z',
   'bid': {'o': '1.11988', 'h': '1.12023', 'l': '1.11788', 'c': '1.11840'},
   'mid': {'o': '1.11995', 'h': '1.12031', 'l': '1.11796', 'c': '1.11852'},
   'ask': {'o': '1.12002', 'h': '1.12039', 'l': '1.11803', 'c': '1.11863'}},
  {'complete': True,
   'volume': 9370,
   'time': '2024-09-27T14:00:00.000000000Z',
   'bid': {'o': '1.11840', 'h': '1.11914', 'l': '1.11684', 'c': '1.11777'},
   'mid': {'o': '1.11852', 'h': '1.11922', 'l': '1.11692', 'c': '1.11785'},
   'ask': {'o': '1.11865', 'h': '1.11929', 'l': '1.11700', 'c': '1.11793'}},

In [47]:
code

200

In [48]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [49]:
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-27T12:00:00.000000000Z,9820,1.11625,1.12018,1.11541,1.11998,1.11617,1.12010,1.11534,1.11990,1.11633,1.12026,1.11548,1.12006
1,2024-09-27T13:00:00.000000000Z,9038,1.11995,1.12031,1.11796,1.11852,1.11988,1.12023,1.11788,1.11840,1.12002,1.12039,1.11803,1.11863
2,2024-09-27T14:00:00.000000000Z,9370,1.11852,1.11922,1.11692,1.11785,1.11840,1.11914,1.11684,1.11777,1.11865,1.11929,1.11700,1.11793
3,2024-09-27T15:00:00.000000000Z,6883,1.11784,1.11784,1.11540,1.11584,1.11777,1.11777,1.11532,1.11577,1.11792,1.11792,1.11548,1.11592
4,2024-09-27T16:00:00.000000000Z,6241,1.11585,1.11600,1.11462,1.11517,1.11577,1.11593,1.11454,1.11509,1.11593,1.11608,1.11470,1.11525
5,2024-09-27T17:00:00.000000000Z,5493,1.11518,1.11658,1.11504,1.11638,1.11510,1.11650,1.11497,1.11630,1.11525,1.11666,1.11512,1.11646
6,2024-09-27T18:00:00.000000000Z,4132,1.11638,1.11678,1.11596,1.11678,1.11630,1.11671,1.11588,1.11671,1.11645,1.11685,1.11604,1.11685
7,2024-09-27T19:00:00.000000000Z,2602,1.11678,1.11679,1.11628,1.11638,1.11670,1.11671,1.11620,1.11631,1.11685,1.11687,1.11635,1.11646
8,2024-09-27T20:00:00.000000000Z,2066,1.11639,1.11680,1.11591,1.11640,1.11631,1.11671,1.11580,1.11625,1.11647,1.11688,1.11599,1.11654


In [50]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [51]:
def create_data(pair,granularity):
    code ,json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair, granularity)

In [52]:
for p1 in our_curr:
    for p2 in our_curr:
        pair =  f"{p1}_{p2}"
        if pair in ins_df.name.unique():
          create_data(pair,"H1")

EUR_USD loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
EUR_GBP loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
EUR_JPY loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
EUR_CHF loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
EUR_NZD loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
EUR_CAD loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
USD_JPY loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
USD_CHF loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
USD_CAD loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
GBP_USD loaded 4000 candles from 2024-02-08T04:00:00.000000000Z to 2024-09-27T20:00:00.000000000Z
GBP_JPY loaded 4000 

In [53]:
ins_df.shape[0]

68

In [54]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,USD_CNH,CURRENCY,USD/CNH,-4,0.05
1,NZD_CAD,CURRENCY,NZD/CAD,-4,0.03
2,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
3,SGD_CHF,CURRENCY,SGD/CHF,-4,0.05
4,EUR_CZK,CURRENCY,EUR/CZK,-4,0.05
...,...,...,...,...,...
63,USD_SEK,CURRENCY,USD/SEK,-4,0.03
64,GBP_SGD,CURRENCY,GBP/SGD,-4,0.05
65,EUR_PLN,CURRENCY,EUR/PLN,-4,0.05
66,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1


In [55]:
ins_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [56]:
ins_df.columns


Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [57]:
ins_df.name

0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object

In [58]:
ins_df.name.unique

<bound method Series.unique of 0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object>